In [2]:
import zipfile
import os

# Specify the directory containing the ZIP files
target_dir = 'E:/bike_Taxi/2023-citibike-tripdata'  # e.g., r'C:\Users\YourName\Downloads'

# Loop through all files in the specified directory
for filename in os.listdir(target_dir):
    if filename.lower().endswith('.zip'):
        zip_path = os.path.join(target_dir, filename)

        # Extract contents directly into the same directory
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(target_dir)
            print(f"Extracted '{filename}' into '{target_dir}'")

print("All ZIP files extracted into their original directory.")


Extracted '202301-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202302-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202303-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202304-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202305-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202306-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202307-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202308-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202309-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202310-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202311-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tripdata'
Extracted '202312-citibike-tripdata.zip' into 'E:/bike_Taxi/2023-citibike-tr

In [3]:
import os
import pandas as pd

# Folder containing your 40 CSV files
folder_path = "E:/bike_Taxi/2023-citibike-tripdata"

# Get all CSV file names
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Store column headers of each file
column_sets = {}

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path, nrows=1)  # Only read first row to get columns
        column_sets[file] = list(df.columns)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Check if all column sets are the same
first_file_columns = list(column_sets.values())[0]

inconsistent_files = [
    fname for fname, cols in column_sets.items()
    if cols != first_file_columns
]

if inconsistent_files:
    print("\n❌ Inconsistent columns found in:")
    for file in inconsistent_files:
        print(f"- {file}")
else:
    print("✅ All CSV files have the same column structure.")


✅ All CSV files have the same column structure.


In [1]:
import os
import pandas as pd

# Set your folder path here
folder_path = "E:/bike_Taxi/2023-citibike-tripdata"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
output_file = os.path.join(folder_path, 'combined_output.csv')

chunk_size = 100000  # Read in manageable chunks
first_file = True  # To write header only once

for f in csv_files:
    file_path = os.path.join(folder_path, f)
    print(f"Processing: {f}")

    try:
        # Try reading in chunks
        for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size, low_memory=False)):
            chunk.to_csv(output_file, mode='a', index=False, header=first_file)
            if first_file:
                first_file = False  # Only write header for the first chunk
    except Exception as e:
        print(f"Failed to process {f}. Error: {e}")

print("✅ All CSVs processed and combined safely without overloading memory.")


Processing: 202301-citibike-tripdata_1.csv
Processing: 202301-citibike-tripdata_2.csv
Processing: 202302-citibike-tripdata_1.csv
Processing: 202302-citibike-tripdata_2.csv
Processing: 202303-citibike-tripdata_1.csv
Processing: 202303-citibike-tripdata_2.csv
Processing: 202303-citibike-tripdata_3.csv
Processing: 202304-citibike-tripdata_1.csv
Processing: 202304-citibike-tripdata_2.csv
Processing: 202304-citibike-tripdata_3.csv
Processing: 202305-citibike-tripdata_1.csv
Processing: 202305-citibike-tripdata_2.csv
Processing: 202305-citibike-tripdata_3.csv
Processing: 202305-citibike-tripdata_4.csv
Processing: 202306-citibike-tripdata_1.csv
Processing: 202306-citibike-tripdata_2.csv
Processing: 202306-citibike-tripdata_3.csv
Processing: 202306-citibike-tripdata_4.csv
Processing: 202307-citibike-tripdata_1.csv
Processing: 202307-citibike-tripdata_2.csv
Processing: 202307-citibike-tripdata_3.csv
Processing: 202307-citibike-tripdata_4.csv
Processing: 202308-citibike-tripdata_1.csv
Processing:

In [1]:
import pandas as pd

file_path = "E:/bike_Taxi/2023-citibike-tripdata/combined_output.csv"

# Read only the header
columns = pd.read_csv(file_path, nrows=0).columns.tolist()

print("📑 Columns in the CSV file:")
print(columns)


📑 Columns in the CSV file:
['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']


In [4]:
with open("E:/bike_Taxi/2023-citibike-tripdata/combined_output.csv") as f:
    row_count = sum(1 for line in f) - 1  # subtract 1 for the header
print(f"Row count: {row_count}")



Row count: 35107030


In [7]:
import pandas as pd
from collections import Counter

file_path = "E:/bike_Taxi/2023-citibike-tripdata/combined_output.csv"
chunk_size = 1_000_000

# First pass — count start station frequencies
station_counter = Counter()
for chunk in pd.read_csv(file_path, chunksize=chunk_size, dtype={'start_station_id': str, 'end_station_id': str}):
    station_counter.update(chunk['start_station_name'].dropna())

# Get top 3 stations
top3_stations = [station for station, _ in station_counter.most_common(3)]
print("Top 3 Start Stations:")
for station in top3_stations:
    print(f"- {station}")

# Second pass — filter only top 3 stations and write output
output_file = 'top3_stations_output.csv'
is_first_chunk = True

for chunk in pd.read_csv(file_path, chunksize=chunk_size, dtype={'start_station_id': str, 'end_station_id': str}):
    filtered_chunk = chunk[chunk['start_station_name'].isin(top3_stations)]
    if not filtered_chunk.empty:
        filtered_chunk.to_csv(output_file, mode='w' if is_first_chunk else 'a', index=False, header=is_first_chunk)
        is_first_chunk = False


Top 3 Start Stations:
- W 21 St & 6 Ave
- Broadway & W 58 St
- West St & Chambers St


In [8]:
with open("E:/bike_Taxi/top3_stations_output.csv") as f:
    row_count = sum(1 for line in f) - 1  # subtract 1 for the header
print(f"Row count: {row_count}")

Row count: 366490


In [9]:
import pandas as pd

file_path = "E:/bike_Taxi/top3_stations_output.csv"

# Read only the header
columns = pd.read_csv(file_path, nrows=0).columns.tolist()

print("📑 Columns in the CSV file:")
print(columns)

📑 Columns in the CSV file:
['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']


In [2]:
import pandas as pd
import numpy as np

# 1. Load CSV
df = pd.read_csv("E:/bike_Taxi/top3_stations_output.csv")  # Replace with your actual filename

# 2. Standardize column names
df.columns = df.columns.str.strip().str.lower()

# 3. Parse datetime columns
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')

# 4. Drop rows with invalid/missing datetime
df = df.dropna(subset=['started_at', 'ended_at'])

# 5. Drop rows with critical missing values
critical_cols = ['ride_id', 'rideable_type', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']
df = df.dropna(subset=critical_cols)

# 6. Fill optional missing values
df['start_station_name'] = df['start_station_name'].fillna('Unknown')
df['end_station_name'] = df['end_station_name'].fillna('Unknown')
df['start_station_id'] = df['start_station_id'].fillna('-1')
df['end_station_id'] = df['end_station_id'].fillna('-1')

# 7. Convert data types
df['ride_id'] = df['ride_id'].astype(str)
df['rideable_type'] = df['rideable_type'].astype('category')
df['member_casual'] = df['member_casual'].astype('category')

# 8. Create ride duration in minutes
df['ride_duration_mins'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
df = df[df['ride_duration_mins'] > 0]  # Remove 0 or negative durations

# 9. Optional time-based features
df['day_of_week'] = df['started_at'].dt.day_name()
df['hour_of_day'] = df['started_at'].dt.hour
df['month'] = df['started_at'].dt.month

# 10. Final check
print(f"✅ Cleaned dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")


✅ Cleaned dataset: 366,225 rows × 17 columns


In [5]:
import os
from dotenv import load_dotenv
import hopsworks

# Load environment variables
load_dotenv()

api_key = os.getenv("HOPSWORKS_API_KEY")
project_name = os.getenv("HOPSWORKS_PROJECT")

# Login to Hopsworks
project = hopsworks.login(
    project=project_name,
    api_key_value=api_key
)

# 1. Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

# 2. Define your cleaned DataFrame (assume it's already created)
# df = your cleaned Citi Bike DataFrame

# 3. Create feature group
from hsfs.feature import Feature
from hsfs.feature_group import FeatureGroup

# Choose primary keys and event time
feature_group = fs.get_or_create_feature_group(
    name="citi_bike_trips",
    version=1,
    description="Cleaned Citi Bike trip data with time features",
    primary_key=["ride_id"],
    event_time="started_at"
)

# 4. Save the DataFrame into the feature group
feature_group.insert(df, write_options={"wait_for_job": True})


2025-05-10 12:56:14,459 INFO: Initializing external client
2025-05-10 12:56:14,461 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 12:56:22,147 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228957
2025-05-10 12:56:22,713 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 12:56:22,719 INFO: Initializing external client
2025-05-10 12:56:22,720 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 12:56:23,847 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228957
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1228957/fs/1213523/fg/1458530


Uploading Dataframe: 100.00% |██████████| Rows 366225/366225 | Elapsed Time: 00:51 | Remaining Time: 00:00


Launching job: citi_bike_trips_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1228957/jobs/named/citi_bike_trips_1_offline_fg_materialization/executions
2025-05-10 12:57:31,086 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-05-10 12:57:34,179 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-05-10 12:59:28,984 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-05-10 12:59:29,248 INFO: Waiting for log aggregation to finish.
2025-05-10 12:59:29,250 INFO: Execution finished successfully.


(Job('citi_bike_trips_1_offline_fg_materialization', 'SPARK'), None)

In [3]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration_mins,day_of_week,hour_of_day,month
0,A46D077151843D7B,classic_bike,2023-01-16 10:39:54.386,2023-01-16 10:45:18.005,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,5.393650,Monday,10,1
1,233875BAED2E02D0,classic_bike,2023-01-12 16:55:30.755,2023-01-12 17:04:03.688,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,8.548883,Thursday,16,1
2,8DD222EA1A1B0BC9,electric_bike,2023-01-08 19:32:25.647,2023-01-08 19:42:00.382,West St & Chambers St,5329.03,West Thames St,5114.06,40.717618,-74.013071,40.708347,-74.017134,member,9.578917,Sunday,19,1
3,58976A4F584F8D28,classic_bike,2023-01-27 20:01:52.897,2023-01-27 20:08:58.118,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,7.087017,Friday,20,1
4,FDD4C1E89A26727C,classic_bike,2023-01-13 18:02:38.160,2023-01-13 18:11:22.139,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,8.732983,Friday,18,1


In [4]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'ride_duration_mins', 'day_of_week', 'hour_of_day',
       'month'],
      dtype='object')